In [1]:
import pennylane as qml
import random
import os
import pandas as pd
import torch
import numpy as np
import matplotlib.pyplot as plt
os.chdir('../')

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Quantum Circuit

In [3]:
date = '20211110_110728'

resume_iters = 30

test_sample_size = 100000

layer =1

qubits = 8

In [4]:
dev = qml.device('default.qubit', wires=qubits)
@qml.qnode(dev, interface='torch')
def gen_circuit(w):
    # random noise as generator input
    z1 = random.uniform(-1, 1)
    z2 = random.uniform(-1, 1)
    
    # construct generator circuit for both atom vector and node matrix
    for i in range(qubits):
        qml.RY(np.arcsin(z1), wires=i)
        qml.RZ(np.arcsin(z2), wires=i)
    for l in range(layer):
        for i in range(qubits):
            qml.RY(w[i], wires=i)
        for i in range(qubits-1):
            qml.CNOT(wires=[i, i+1])
            qml.RZ(w[i+qubits], wires=i+1)
            qml.CNOT(wires=[i, i+1])
    return [qml.expval(qml.PauliZ(i)) for i in range(qubits)]

In [5]:
model_dir_path = r'/home/ken/projects/QuantumGAN-PyTorch/results/quantum-GAN/'+date+'/train/model_dir'

weights_pth = os.path.join(model_dir_path, 'molgan_red_weights.csv')

weights = pd.read_csv(weights_pth, header=None).iloc[resume_iters-1, 1:].values

gen_weights = torch.tensor(list(weights), requires_grad=True)

In [6]:
sample_list = [gen_circuit(gen_weights) for i in range(test_sample_size)]

In [7]:
sample_list[0]

tensor([-0.6315,  0.6767,  0.4883, -0.4562,  0.6796, -0.5655,  0.7679,  0.6602],
       dtype=torch.float64, grad_fn=<CatBackward>)

# Data

In [8]:
from data.sparse_molecular_dataset import SparseMolecularDataset

In [9]:
mol_data_dir = 'data/gdb9_9nodes.sparsedataset'

In [10]:
data = SparseMolecularDataset()

data.load(mol_data_dir)

# Generator

In [11]:
from models.models import Generator, Discriminator

In [12]:
complexity = 'mr'

z_dim = qubits

dropout = 0.0

In [13]:
if complexity == 'nr':
    g_conv_dim = [128, 256, 512]
elif complexity == 'mr':
    g_conv_dim = [128]
elif complexity == 'hr':
    g_conv_dim = [16]
else:
    raise ValueError("Please enter an valid model complexity from 'mr', 'hr' or 'nr'!")

In [14]:
G = Generator(g_conv_dim, z_dim, data.vertexes, data.bond_num_types, data.atom_num_types, dropout)

In [15]:
G.to(device)

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [ ]:
G_path = os.path.join(model_dir_path, '{}-G.ckpt'.format(resume_iters))

In [ ]:
G.load_state_dict(torch.load(G_path, map_location=lambda storage, loc: storage))

# Generating Molecules

In [ ]:
import torch.nn.functional as F

In [ ]:
z = torch.stack(tuple(sample_list)).to(device).float()

In [ ]:
post_method = 'softmax'

In [ ]:
edges_logits, nodes_logits = G(z)

In [ ]:
def postprocess(inputs, method, temperature=1.0):
    def listify(x):
        return x if type(x) == list or type(x) == tuple else [x]
    def delistify(x):
        return x if len(x) > 1 else x[0]
    if method == 'soft_gumbel':
        softmax = [F.gumbel_softmax(e_logits.contiguous().view(-1, e_logits.size(-1))/temperature, hard=False).view(e_logits.size()) for e_logits in listify(inputs)]
    elif method == 'hard_gumbel':
        softmax = [F.gumbel_softmax(e_logits.contiguous().view(-1, e_logits.size(-1))/temperature, hard=True).view(e_logits.size()) for e_logits in listify(inputs)]
    else:
        softmax = [F.softmax(e_logits/temperature, -1) for e_logits in listify(inputs)]
    return [delistify(e) for e in (softmax)]

In [ ]:
(edges_hat, nodes_hat) = postprocess((edges_logits, nodes_logits), post_method)   

In [ ]:
def get_gen_mols(data, n_hat, e_hat, method):
    (edges_hard, nodes_hard) = postprocess((e_hat, n_hat), method)
    edges_hard, nodes_hard = torch.max(edges_hard, -1)[1], torch.max(nodes_hard, -1)[1]
    mols = [data.matrices2mol(n_.data.cpu().numpy(), e_.data.cpu().numpy(), strict=True) for e_, n_ in zip(edges_hard, nodes_hard)]
    return mols

In [ ]:
mols = get_gen_mols(data, nodes_logits, edges_logits, post_method)

# Scores

In [ ]:
from utils.utils import *

In [ ]:
m0, m1 = all_scores(mols, data, norm=True)

In [ ]:
from collections import defaultdict

scores = defaultdict(list)

for k, v in m1.items():
    scores[k].append(v)
for k, v in m0.items():
    scores[k].append(np.array(v)[np.nonzero(v)].mean())

In [ ]:
scores

In [ ]:
v = MolecularMetrics.valid_filter(mols)

In [ ]:
s = set(map(lambda x: Chem.MolToSmiles(x), v))

In [ ]:
print(len(s))

In [ ]:
from rdkit.Chem import Draw

unique_mols = list(map(lambda x: Chem.MolFromSmiles(x), s))

In [ ]:
img=Draw.MolsToGridImage(unique_mols,molsPerRow=8,subImgSize=(200,200), maxMols=200)    

In [ ]:
img

In [ ]:
df = pd.DataFrame({'SMIELS': data} for data in s)

In [ ]:
df.to_csv('quantum-'+str(resume_iters)+'.csv', index=False)